In [3]:
def update_1d(w, err, jac, eta = 1, q_=0):
    update = jac ** (1 - q) * err
    w -= eta * update
    return w

def update_2d(w, err, jac, eta = 1, q_=0):
    U, A, V = svd_2d(jac)
    update = U * (A ** (1 - q)) @ V @ err
    w -= eta * update
    return w

In [5]:
def get_orth_matrix(S):
    theta   = 0.5*np.arctan2(S[0,1]+S[1,0], S[0,0]-S[1,1])   
    Cos, Sin = np.cos(theta), np.sin(theta)
    return np.array([[Cos, -Sin], [Sin, Cos]])
    
def svd_2d(A):   #   SVD of 2x2 matrix A:  U and V are orthogonal, D is diagonal, and A=U*D@V
    U = get_orth_matrix(A @ A.T)
    V = get_orth_matrix(A.T @ A)
    d = np.diag(U.T @ A @ V)
    
    # sign correction
    V = np.sign(d) * V
    d = np.abs(d)
    
    return U, d, V.T

# A= np.random.randn(2,2)
# U, d, V =svd_2d(A)
# print(A - U * d @ V)